首先 import 必要的模块

In [1]:
import lightgbm as lgbm
#from lgbm.sklearn import LGBMClassifier
import xlearn as xl
import pandas as pd 
import numpy as np
import time

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss     #采用logloss作为评价指标
from pandas import DataFrame

from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder
from matplotlib import pyplot
import seaborn as sns
%matplotlib inline
from libffm_data_generate import *

## 1、 数据输入

### 1.1读取数据

In [2]:
train_name_1='100_0'
train_name_2='100_1'
valid_name_1='100_2'
valid_name_2='100_3'

In [3]:
#train_data
train_1 = pd.read_csv("/home/jupyter/训练数据组/100w数据组/"+train_name_1,index_col=0).reset_index(drop=True)   #拿200万条数据训练
train_1.drop(['id','hour','user_id','Unnamed: 0'],axis=1,inplace=True)
print(train_1.shape)
#validation_data
valid_1 = pd.read_csv("/home/jupyter/训练数据组/100w数据组/"+valid_name_1,index_col=0).reset_index(drop=True)   #拿200万条数据训练
valid_1.drop(['id','hour','user_id','Unnamed: 0'],axis=1,inplace=True)
print(valid_1.shape)

(999864, 23)
(999864, 23)


In [11]:
#train_data
train_2 = pd.read_csv("/home/jupyter/训练数据组/100w数据组/"+train_name_2,index_col=0).reset_index(drop=True)   #拿200万条数据训练
train_2.drop(['id','hour','user_id','Unnamed: 0'],axis=1,inplace=True)
print(train_2.shape)
valid_2 = pd.read_csv("/home/jupyter/训练数据组/100w数据组/"+valid_name_2,index_col=0).reset_index(drop=True)   #拿200万条数据训练
valid_2.drop(['id','hour','user_id','Unnamed: 0'],axis=1,inplace=True)
print(valid_2.shape)

(999864, 23)
(996224, 23)


In [4]:
#test_data
test = pd.read_csv("/home/jupyter/训练数据组/FE_test_26_final.csv",index_col=0).reset_index(drop=True)   #拿200万条数据训练
test.drop(['id','hour','user_id'],axis=1,inplace=True)
test.shape

/home/qiaoliang/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(4218938, 23)

In [5]:
#first
y_train_1 = train_1['click']
x_train_1 = train_1.drop(["click"], axis=1)

y_valid_1 = valid_1['click']
x_valid_1 = valid_1.drop(["click"], axis=1)

In [6]:
#second
y_train_2 = train_2['click']
x_train_2 = train_2.drop(["click"], axis=1)

y_valid_2 = valid_2['click']
x_valid_2 = valid_2.drop(["click"], axis=1)

NameError: name 'train_2' is not defined

In [9]:
y_test = test['click']
x_test = test.drop(["click"], axis=1)

### 1.2 原始特征直接生成供ffm格式数据

In [11]:
#ffm-data
#train
start=time.time()
original_ffm_data(x=x_train_1,y=y_train_1,save_name=r'/home/jupyter/xlearn/ffm_'+train_name_1+'.txt')
end=time.time()
print(end-start)
#valid
start=time.time()
original_ffm_data(x=x_valid_1,y=y_valid_1,save_name=r'/home/jupyter/xlearn/ffm_'+valid_name_1+'.txt')
end=time.time()
print(end-start)

1314.798838853836
1127.893979549408


In [41]:
#test
start=time.time()
original_ffm_data(x=x_test,y=y_test,save_name=r'/home/jupyter/xlearn/'+'ffm_test.txt')
end=time.time()
print(end-start)

174.09121370315552
173.4769024848938
754.1248562335968


In [32]:
#fm_data
x_train_fm_1=np.array(x_train_1)
x_valid_fm_1=np.array(x_valid_1)
x_test_fm_1=np.array(x_test)
xl.write_data_to_xlearn_format(X=x_train_fm_1,y=y_train_1,filepath=r'/home/jupyter/xlearn/fm'+train_1_name+'.txt')
xl.write_data_to_xlearn_format(X=x_valid_fm_1,y=y_valid_1,filepath=r'/home/jupyter/xlearn/fm'+valid_name_1+'.txt')
xl.write_data_to_xlearn_format(X=x_test_fm_1,y=y_test,filepath=r'/home/jupyter/xlearn/fm'+'fm_test.txt')

In [30]:
del train_1
del train_2
del valid_1
del valid_2
del test

默认参数，此时学习率为0.1，比较大，观察弱分类数目的大致范围
（采用默认参数配置，看看模型是过拟合还是欠拟合）

## 2. lightgbm 训练

In [10]:
start=time.time()
# 定义模型
lgb1 = lgbm.LGBMRegressor(
          num_boost_round=20,
          boosting='gbdt', 
          objective='binary', 
          num_threads=6, 
          #default=OpenMP_default, type=int, alias=num_thread, nthread LightGBM 的线程数
#          'silent': True,
          learning_rate=0.1, 
          num_leaves=10, 
          #num_leaves：叶子数
#num_leaves, default=31, type=int, alias=num_leaf一棵树上的叶子数
          max_depth= 4,
          max_bi= 256, 
          subsample_for_bin= 200000,#默认200000，用来构建直方图数据数量
          subsample= 0.8, #默认1
          #类似于feature_fraction,它将在不进行重采样的情况下
          #随机选择部分数据可以用来加速训练可以用来处理过拟合Not
          subsample_freq= 1, 
          colsample_bytree= 0.8,       
          reg_alpha= 0.8,         
          reg_lambda= 0.25, 
          min_split_gain=0.0, 
          min_child_weight= 1,        
          min_child_samples= 200, 
          min_data_in_leaf=50, 
          #default=20, type=int, alias=min_data_per_leaf , min_data, 
          #min_child_samples一个叶子上数据的最小数量. 可以用来处理过拟合.
        
        # 'categorical_feature':3
         # 'categorical_feature':3:26
         )
# 训练学习
model=lgb1.fit(x_train_1, y_train_1)
end=time.time()
print(end-start)

/home/qiaoliang/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


1.0086908340454102


In [11]:
# train上logloss评测
y_pred_train = lgb1.predict(x_train_1)
logloss = log_loss(y_train_1, y_pred_train)
print ("logloss of train :",logloss )

# valid上logloss评测
y_pred_valid = lgb1.predict(x_valid_1)
logloss = log_loss(y_valid_1, y_pred_valid)
print ("logloss of valid :",logloss )

# test上logloss评测
y_pred_test = lgb1.predict(x_test)
logloss = log_loss(y_test, y_pred_test)
print ("logloss of test :",logloss )

logloss of train : 0.4282512679422011
logloss of valid : 0.4284860147117886
logloss of test : 0.42647534996541225


##### 第一组训练-验证数据

In [13]:
x_train_leaves = lgb1.predict(x_train_1, pred_leaf=True)    
x_valid_leaves = lgb1.predict(x_valid_1, pred_leaf=True) 
x_test_leaves = lgb1.predict(x_test, pred_leaf=True) 
print(x_train_leaves.shape)
print(x_valid_leaves.shape)
print(x_test_leaves.shape)

(999864, 20)
(999864, 20)
(4218938, 20)


##### 第二组训练-验证数据

In [9]:
x_train_leaves = lgb1.predict(x_train_2, pred_leaf=True)    
x_valid_leaves = lgb1.predict(x_valid_2, pred_leaf=True) 
print(x_train_leaves.shape)
print(x_valid_leaves.shape)

(999864, 50)
(996224, 50)


## 3、x-learn train

### 3.1 lightgbm输出叶子节点数据转化为libffm或liffm格式数据

##### 生成fm数据

In [16]:
#用于gbm训练的数据，直接进入fm
# self def to produce libffm data
start=time.time()
xl.write_data_to_xlearn_format(X=x_train_leaves,y=y_train_1,filepath=r'/home/jupyter/xlearn/lightgbm_ffm/fm_train.txt')
xl.write_data_to_xlearn_format(X=x_valid_leaves,y=y_valid_1,filepath=r'/home/jupyter/xlearn/lightgbm_ffm/fm_valid.txt')
xl.write_data_to_xlearn_format(X=x_test_leaves,y=y_test,filepath=r'/home/jupyter/xlearn/lightgbm_ffm/fm_test.txt')
end=time.time()
print(end-start)

183.07247114181519


#### ffm

In [12]:
#用于gbm训练的数据，直接进入ffm
# self def to produce libffm data
start=time.time()
gbm_ffm_data(x=x_train_leaves,y=y_train_1,save_name=r'/home/jupyter/xlearn/lightgbm_ffm/ffm_train.txt')
gbm_ffm_data(x=x_valid_leaves,y=y_valid_1,save_name=r'/home/jupyter/xlearn/lightgbm_ffm/ffm_valid.txt')
gbm_ffm_data(x=x_test_leaves,y=y_test,save_name=r'/home/jupyter/xlearn/lightgbm_ffm/ffm_test.txt')
end=time.time()
print(end-start)


NameError: name 'x_train_leaves' is not defined

### 3.2 模型训练

##### 原始数据的ffm模型初始化

In [5]:
# original Training task
ffm_model = xl.create_ffm() # Use field-aware factorization machine
ffm_model.setTrain(r'/home/jupyter/xlearn/ffm_'+train_name_1+'.txt')  # Training data
ffm_model.setValidate(r'/home/jupyter/xlearn/ffm_'+valid_name_1+'.txt')  # Validation data
ffm_model.setTest(r'/home/jupyter/xlearn/ffm_test.txt')  # test data

##### lightgbm+ffm模型初始化

In [14]:
# lightgbm Training task
ffm_model = xl.create_ffm() # Use field-aware factorization machine
ffm_model.setTrain(r'/home/jupyter/xlearn/lightgbm_ffm/ffm_train.txt')  # Training data
ffm_model.setValidate(r'/home/jupyter/xlearn/lightgbm_ffm/ffm_valid.txt')  # Validation data
ffm_model.setTest(r'/home/jupyter/xlearn/lightgbm_ffm/ffm_test.txt') 

##### 模型训练

In [15]:
# param:
#  0. binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: accuracy
start=time.time()
param = {'task':'binary','k':4,'opt':'adagrad','lr':0.2,'lambda':0.00003, 
         'metric':'acc','fold':2,'epoch':30,'stop_window':3}
ffm_model.setSign()  # Convert output to 0-1  scope
result=ffm_model.fit(param, r'/home/jupyter/xlearn/model.out')
result_cv=ffm_model.cv(param)
# Prediction task
end=time.time()
print(end-start)
time.sleep(5)
ffm_model.predict(r'/home/jupyter/xlearn/model.out', r'/home/jupyter/xlearn/output.txt')
#0.4148

70.34164953231812


In [17]:
# param:
#  0. binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: accuracy
start=time.time()
param = {'task':'binary','k':4,'opt':'adagrad','lr':0.3,'lambda':0.00003, 
         'metric':'acc','fold':2,'epoch':30,'stop_window':3}
ffm_model.setSign()  # Convert output to 0-1  scope
result=ffm_model.fit(param, r'/home/jupyter/xlearn/model.out')
result_cv=ffm_model.cv(param)
# Prediction task
end=time.time()
print(end-start)
time.sleep(5)
ffm_model.predict(r'/home/jupyter/xlearn/model.out', r'/home/jupyter/xlearn/output.txt')
#0.41507

67.06751322746277
